In [1]:
import pandas as pd
from sklearn import *
from datetime import datetime

In [2]:
def preprocess(x):
    x = x.dropna()

    dates = [datetime.strptime(i, '%Y-%m-%d') for i in x['Date'].to_list()]

    x['year'] = [int(date.year) for date in dates]
    x['month'] = [int(date.month) for date in dates]
    # Day of month doesn't really seem to help, the day of the year is more helpful
    x['day'] = [int(date.timetuple().tm_yday) for date in dates]

    # x['type'] = [1 if row == 'organic' else 0 for row in x['type']]

    # One Hot
    x = pd.merge(x, pd.get_dummies(x['region'], prefix='region'), how='right', left_index=True, right_index=True)

    x = x.drop('Date', axis=1)
    # Somehow dropping type makes it more accurate
    x = x.drop('type', axis=1)
    x = x.drop('region', axis=1)

    return x

# Train Model

In [3]:
df = pd.read_csv('avocado-train.csv', index_col='id')
df.head()

,DataBatch,Date,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,region,AveragePrice
id,,,,,,,,,,,,,
0,7,2016-11-06,183542.31,98949.98,22891.61,95.00,61605.72,43571.99,17499.01,534.72,conventional,NewOrleansMobile,1.49
1,24,2017-07-16,224434.92,42951.31,120360.02,131.85,60991.74,53141.81,3621.04,4228.89,conventional,HarrisburgScranton,1.38
2,51,2015-01-04,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,organic,Atlanta,1.76
3,22,2015-07-26,91825.07,1679.28,45615.48,741.77,43788.54,43788.54,0.00,0.00,conventional,BuffaloRochester,1.39
4,50,2015-01-11,54644.32,1491.88,33759.12,1325.17,18068.15,12165.94,5902.21,0.00,conventional,Pittsburgh,1.54


In [4]:
df_processed = preprocess(df)

features = df_processed.columns.values.tolist()
features.remove('AveragePrice')
x = df_processed[features]
y = df_processed['AveragePrice']
x.head()

,DataBatch,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,year,month,day,region_Albany,region_Atlanta,region_BaltimoreWashington,region_Boise,region_Boston,region_BuffaloRochester,region_California,region_Charlotte,region_Chicago,region_CincinnatiDayton,region_Columbus,region_DallasFtWorth,region_Denver,region_Detroit,region_GrandRapids,region_GreatLakes,region_HarrisburgScranton,region_HartfordSpringfield,region_Houston,region_Indianapolis,region_Jacksonville,region_LasVegas,region_LosAngeles,region_Louisville,region_MiamiFtLauderdale,region_Midsouth,region_Nashville,region_NewOrleansMobile,region_NewYork,region_Northeast,region_NorthernNewEngland,region_Orlando,region_Philadelphia,region_PhoenixTucson,region_Pittsburgh,region_Plains,region_Portland,region_RaleighGreensboro,region_RichmondNorfolk,region_Roanoke,region_Sacramento,region_SanDiego,region_SanFrancisco,region_Seattle,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,7,183542.31,98949.98,22891.61,95.00,61605.72,43571.99,17499.01,534.72,2016,11,311,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,24,224434.92,42951.31,120360.02,131.85,60991.74,53141.81,3621.04,4228.89,2017,7,197,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,51,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,2015,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,22,91825.07,1679.28,45615.48,741.77,43788.54,43788.54,0.00,0.00,2015,7,207,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,54644.32,1491.88,33759.12,1325.17,18068.15,12165.94,5902.21,0.00,2015,1,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.2)
print('Train split:', len(x_train))
print('Test split:', len(x_val))
print('Total:', len(x),len(y))

Train split: 10219
Test split: 2555
Total: 12774 12774


In [6]:
cpu_cores = !nproc
# model = ensemble.RandomForestRegressor(random_state=2021, n_estimators=500, n_jobs=int(cpu_cores[0]) + 1)
model = ensemble.ExtraTreesRegressor(random_state=2021, n_estimators=500, n_jobs=int(cpu_cores[0]) + 1)

In [7]:
# Use the full dataset when submitting to server
model.fit(x, y)
# model.fit(x_train, y_train)
# model.score(x_val, y_val)
load_avg = !cat /proc/loadavg | cut -f1 -d' '
print('Load Average:', load_avg[0])

Load Average: 1.53


# Prediction

In [8]:
df_test = pd.read_csv('avocado-test.csv', index_col='id')
df_test.head()

,DataBatch,Date,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,region
id,,,,,,,,,,,,
0,21,2016-07-31,28969.34,80.77,27361.91,0.00,1526.66,1526.66,0.00,0.00,organic,Chicago
1,19,2017-08-20,322962.89,5632.85,242365.82,2441.56,72522.66,22942.48,46083.51,3496.67,conventional,NorthernNewEngland
2,49,2017-01-22,155334.45,3657.79,74068.65,0.00,77608.01,27302.29,50305.72,0.00,organic,GreatLakes
3,12,2015-10-04,10231.74,341.89,8519.00,0.00,1370.85,303.33,1067.52,0.00,organic,CincinnatiDayton
4,10,2015-10-18,10652.02,6905.95,39.52,0.00,3706.55,3706.55,0.00,0.00,organic,Houston


In [9]:
df_test_processed = preprocess(df_test)
x_test = df_test_processed[features]
x_test = x_test.dropna()
print('Rows:', len(x_test))
x_test.head()

Rows: 5475


,DataBatch,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,XLargeBags,year,month,day,region_Albany,region_Atlanta,region_BaltimoreWashington,region_Boise,region_Boston,region_BuffaloRochester,region_California,region_Charlotte,region_Chicago,region_CincinnatiDayton,region_Columbus,region_DallasFtWorth,region_Denver,region_Detroit,region_GrandRapids,region_GreatLakes,region_HarrisburgScranton,region_HartfordSpringfield,region_Houston,region_Indianapolis,region_Jacksonville,region_LasVegas,region_LosAngeles,region_Louisville,region_MiamiFtLauderdale,region_Midsouth,region_Nashville,region_NewOrleansMobile,region_NewYork,region_Northeast,region_NorthernNewEngland,region_Orlando,region_Philadelphia,region_PhoenixTucson,region_Pittsburgh,region_Plains,region_Portland,region_RaleighGreensboro,region_RichmondNorfolk,region_Roanoke,region_Sacramento,region_SanDiego,region_SanFrancisco,region_Seattle,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,21,28969.34,80.77,27361.91,0.00,1526.66,1526.66,0.00,0.00,2016,7,213,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19,322962.89,5632.85,242365.82,2441.56,72522.66,22942.48,46083.51,3496.67,2017,8,232,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,49,155334.45,3657.79,74068.65,0.00,77608.01,27302.29,50305.72,0.00,2017,1,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,12,10231.74,341.89,8519.00,0.00,1370.85,303.33,1067.52,0.00,2015,10,277,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10,10652.02,6905.95,39.52,0.00,3706.55,3706.55,0.00,0.00,2015,10,291,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
y_test = model.predict(x_test)
output = pd.DataFrame({'id': x_test.index, 'AveragePrice': y_test})
output.to_csv('avocado-submission.csv', index=False)

# Submission

In [11]:
import pyctfsglib as ctfsg
import random

with open('token') as f:
    USER_TOKEN = f.read()
GRADER_URL = random.choice([
    'http://3qo9k5hk5cprtqvnlkvotlnj9d14b7mt.ctf.sg:60010/',
    'http://3qo9k5hk5cprtqvnlkvotlnj9d14b7mt.ctf.sg:60011/'
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)

DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm AVOCADO_PRICE @bc6e1a3c2caa


In [12]:
grader.submitFile('avocado-submission.csv')

ProofOfWork Challenge =>  ('CTFSGRBa1c5118c6330e6a248c19e0161db777e', 22)
ProofOfWork Answer Found! =>  1590430


'{"challenge":{"name":"AvocadoRegression"},"id":"cknbih018c5n00824w49nb1ca","status":"PARTIALLY_CORRECT","multiplier":0.9423,"submittedBy":{"username":"hci-19"},"createdAt":"2021-04-10T09:03:49Z"}'